# Solving 3-SAT Problem with Hybrid Quantum-Classical solver 

**Author:** Xinghui Jia \& Siwei Tan

**Date:** 9/4/2024

Based on paper "[Janus-CT: A Hybrid Approach for 3-SAT Problems by Integrating Quantum Annealer with CDCL][1]"

[1]: https://ieeexplore.ieee.org/abstract/document/10071022/


A propositional satisfiability (SAT) problem is to find an assignment for each variable to satisfy a given Boolean formula. A typical SAT problem is composed of multiple clauses represented as a disjunction of Boolean variables. 3-SAT problem is a special case of the SAT problem that has no more than three variables in each clause. 

3-SAT problem is a fundamental problem in various applications, such as artificial intelligence, circuit analysis, and protein structure prediction. Since the 3-SAT problem is an NP-complete problem, the time complexity of classical algorithms increases exponentially with the number of clauses and variables. We observe that the classical CDCL algorithm has the advantage of solving larger problems, while QA is more suitable for solving small but hard problems. To combine their advantages, we introduce Janus-SAT in this notebook, a hybrid approach that integrates quantum annealing (QA) with the classical Conflict-Driven Clause Learning (CDCL) algorithm to enable end-to-end acceleration for solving 3-SAT problems. 


<div style="text-align:center;">
    <img src="./pictures/4-1.hyqsat.png"  width="45%" height="45%">
</div>

The figure above shows the workflow of Janus-CT. It features a cross-iterative process. The CDCL algorithm searches for the solution. During the search, it identifies and sends the hard sub-problem to QA. Janus-CT designed a fast compilation flow to embed the problem and optimize the noise. It also designs multiple strategies to guide the search of CDCL using the QA results.

In [1]:
import sys
sys.path.append('..')

from janusq.hyqsat import solve_by_janusct, solve_by_minisat

In [2]:
# Configure the model
file_path = "cnf_examples/uf50-01.cnf"  # input cnf flie
cpu_lim = 0  # limit the cpu time (s). 0 means infinite
mem_lim = 0  # limit the memory 0 means infinite
strictp = True  # isStrict

### Using Janus-SAT API to Solve the Problem

We can use a QA simulator or real-world QA hardware to solve the SAT roblem. For example, here we use the simualtor.

In [8]:
# solve by Janus-SAT
solve_by_janusct(file_path, verb=False, cpu_lim=cpu_lim, mem_lim=mem_lim)

2024-04-16 11:48:36,408 - INFO: WARNING: for repeatability, setting FPU to use double precision
2024-04-16 11:48:36,408 - INFO: sys.path.append('/home/JanusQ-main/examples/../janusq/hyqsat/python/')
2024-04-16 11:48:36,409 - INFO: Quantum method:
2024-04-16 11:48:36,409 - INFO: /home/JanusQ-main/examples/cnf_examples/uf50-01.cnf


2024-04-16 11:48:37,618 - INFO: SATISFIABLE
2024-04-16 11:48:37,619 - INFO: SAT
2024-04-16 11:48:37,619 - INFO: -1 2 -3 4 5 6 7 8 9 -10 -11 12 -13 14 -15 -16 -17 -18 19 20 -21 -22 23 -24 -25 -26 27 -28 -29 -30 -31 -32 -33 -34 35 36 37 -38 39 -40 -41 -42 -43 -44 -45 -46 -47 48 49 -50 0
2024-04-16 11:48:37,619 - INFO: ^0 1 ^2 3 4 5 6 7 8 ^9 ^10 11 ^12 13 ^14 ^15 ^16 ^17 18 19 ^20 ^21 22 ^23 ^24 ^25 26 ^27 ^28 ^29 ^30 ^31 ^32 ^33 34 35 36 ^37 38 ^39 ^40 ^41 ^42 ^43 ^44 ^45 ^46 47 48 ^49
2024-04-16 11:48:37,620 - INFO: 
2024-04-16 11:48:37,620 - INFO: 
2024-04-16 11:48:37,621 - INFO: /home/JanusQ-main/examples/cnf_examples/uf50-01.cnf has been solved.


{'isSatisfiable': True, 'isSat': True}

Note that when using the simulator, the solving time is estimated as (CDCL time + number of QA $\times$ 120 $\mu s$).

### Comparision to MiniSAT

We compare it to the MiniSAT solver.

In [10]:
# solve by minisat
solve_by_minisat(file_path, verb=True, cpu_lim=cpu_lim, mem_lim=mem_lim)

2024-04-16 11:49:46,178 - INFO: WARNING: for repeatability, setting FPU to use double precision
2024-04-16 11:49:46,179 - INFO: Original method:
2024-04-16 11:49:46,179 - INFO: /home/JanusQ-main/examples/cnf_examples/uf50-01.cnf
2024-04-16 11:49:46,233 - INFO: ============================[ Problem Statistics ]=============================
2024-04-16 11:49:46,234 - INFO: |                                                                             |
2024-04-16 11:49:46,234 - INFO: |  Number of variables:            50                                         |
2024-04-16 11:49:46,234 - INFO: |  Number of clauses:             218                                         |
2024-04-16 11:49:46,235 - INFO: ============================[ Search Statistics ]==============================
2024-04-16 11:49:46,235 - INFO: | Conflicts |          ORIGINAL         |          LEARNT          | Progress |
2024-04-16 11:49:46,235 - INFO: |           |    Vars  Clauses Literals |    Limit  Clauses Lit/Cl 

{'restarts': 1,
 'conflicts': 16,
 'conflict cost': 0.107,
 'decisions': 0,
 'propagations': 0,
 'conflict literals': 65,
 'actual CPU time': 0.057791,
 'this problem time': 0.47,
 'annealing time': 0.0,
 'quantum count': 0,
 'simulate time': 0.0,
 'quantum success number': 0,
 'quantum conflict number': 0,
 'quantum one time solve number': 0,
 'isSatisfiable': True,
 'isSat': True}